此示例显示可以组合因子以创建新因子，可以使用`mask`参数对因子进行排名，以及如何使用`pipeline`API创建多头和空头列表。

In [1]:
%load_ext zipline

/home/ldf/anaconda3/envs/zipline/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
%%zipline --start 2017-1-1 --end 2017-12-31

from logbook import Logger
from zipline.api import attach_pipeline, pipeline_output
from zipline.pipeline import Pipeline
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.factors import SimpleMovingAverage

logger = Logger('回测')

def initialize(context):

    sma_short = SimpleMovingAverage(
        inputs=[USEquityPricing.close], window_length=30)
    sma_long = SimpleMovingAverage(
        inputs=[USEquityPricing.close], window_length=100)

    # Combined factors to create new factors
    sma_quotient = sma_short / sma_long

    # Create a screen to remove penny stocks
    remove_penny_stocks = sma_short > 1.0

    sma_rank = sma_quotient.rank(mask=remove_penny_stocks)

    # Rank a factor using a mask to ignore the values we're
    # filtering out by passing mask=remove_penny_stocks to rank.
    longs = sma_rank.top(200, mask=remove_penny_stocks)
    shorts = sma_rank.bottom(200, mask=remove_penny_stocks)

    pipe_columns = {'longs': longs, 'shorts': shorts, 'sma_rank': sma_rank}

    # Use multiple screens to narrow the universe
    pipe_screen = (longs | shorts)

    pipe = Pipeline(columns=pipe_columns, screen=pipe_screen)
    attach_pipeline(pipe, 'example')


def before_trading_start(context, data):
    context.output = pipeline_output('example')

    context.longs = context.output[context.output.longs]
    context.shorts = context.output[context.output.shorts]

    context.security_list = context.shorts.index.union(context.longs.index)

    # Print the 5 securities with the lowest sma_rank.
    logger.info("SHORT LIST")
    logger.info(
        "\n" + str(context.shorts.sort_values(['sma_rank'], ascending=True).head()))

    # Print the 5 securities with the highest sma_rank.
    logger.info("LONG LIST")
    logger.info(
        "\n" + str(context.longs.sort_values(['sma_rank'], ascending=False).head()))

[2018-05-09 07:53:56.637412] INFO: 回测: SHORT LIST
[2018-05-09 07:53:56.642885] INFO: 回测: 
              longs  shorts  sma_rank
东方网络(002175)  False    True       1.0
巨人网络(002558)  False    True       2.0
思美传媒(002712)  False    True       3.0
天润曲轴(002283)  False    True       4.0
金石东方(300434)  False    True       5.0
[2018-05-09 07:53:56.644013] INFO: 回测: LONG LIST
[2018-05-09 07:53:56.649014] INFO: 回测: 
              longs  shorts  sma_rank
三江购物(601116)   True   False    3029.0
上海凤凰(600679)   True   False    3028.0
齐翔腾达(002408)   True   False    3027.0
卓郎智能(600545)   True   False    3026.0
南洋科技(002389)   True   False    3025.0
[2018-05-09 07:53:56.654958] INFO: 回测: SHORT LIST
[2018-05-09 07:53:56.662405] INFO: 回测: 
              longs  shorts  sma_rank
东方网络(002175)  False    True       1.0
巨人网络(002558)  False    True       2.0
天润曲轴(002283)  False    True       3.0
金石东方(300434)  False    True       4.0
思美传媒(002712)  False    True       5.0
[2018-05-09 07:53:56.664377] INFO: 回测: LONG LIS

/home/ldf/anaconda3/envs/zipline/lib/python3.6/site-packages/empyrical-0.4.2-py3.6.egg/empyrical/stats.py:676: RuntimeWarning: invalid value encountered in true_divide
  out=out,
/home/ldf/anaconda3/envs/zipline/lib/python3.6/site-packages/empyrical-0.4.2-py3.6.egg/empyrical/stats.py:762: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)


[2018-05-09 07:53:57.351394] INFO: 回测: SHORT LIST
[2018-05-09 07:53:57.357403] INFO: 回测: 
               longs  shorts  sma_rank
东方网络(002175)   False    True       1.0
巨人网络(002558)   False    True       2.0
科大国创(300520)   False    True       3.0
天润曲轴(002283)   False    True       4.0
*ST烯碳(000511)  False    True       5.0
[2018-05-09 07:53:57.358785] INFO: 回测: LONG LIST
[2018-05-09 07:53:57.365300] INFO: 回测: 
              longs  shorts  sma_rank
三江购物(601116)   True   False    3045.0
南洋科技(002389)   True   False    3044.0
上海凤凰(600679)   True   False    3043.0
卓郎智能(600545)   True   False    3042.0
齐翔腾达(002408)   True   False    3041.0
[2018-05-09 07:53:57.372343] INFO: 回测: SHORT LIST
[2018-05-09 07:53:57.378967] INFO: 回测: 
               longs  shorts  sma_rank
东方网络(002175)   False    True       1.0
巨人网络(002558)   False    True       2.0
科大国创(300520)   False    True       3.0
天润曲轴(002283)   False    True       4.0
*ST烯碳(000511)  False    True       5.0
[2018-05-09 07:53:57.380113] INFO: 

/home/ldf/anaconda3/envs/zipline/lib/python3.6/site-packages/empyrical-0.4.2-py3.6.egg/empyrical/stats.py:676: RuntimeWarning: invalid value encountered in true_divide
  out=out,
/home/ldf/anaconda3/envs/zipline/lib/python3.6/site-packages/empyrical-0.4.2-py3.6.egg/empyrical/stats.py:762: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)


[2018-05-09 07:53:57.576018] INFO: 回测: 
              longs  shorts  sma_rank
东方网络(002175)  False    True       1.0
金冠电气(300510)  False    True       2.0
科大国创(300520)  False    True       3.0
天润曲轴(002283)  False    True       4.0
巨人网络(002558)  False    True       5.0
[2018-05-09 07:53:57.577307] INFO: 回测: LONG LIST
[2018-05-09 07:53:57.582755] INFO: 回测: 
              longs  shorts  sma_rank
三江购物(601116)   True   False    3079.0
江苏国信(002608)   True   False    3078.0
中航沈飞(600760)   True   False    3077.0
贝瑞基因(000710)   True   False    3076.0
柘中股份(002346)   True   False    3075.0
[2018-05-09 07:53:57.589019] INFO: 回测: SHORT LIST
[2018-05-09 07:53:57.593843] INFO: 回测: 
              longs  shorts  sma_rank
东方网络(002175)  False    True       1.0
金冠电气(300510)  False    True       2.0
科大国创(300520)  False    True       3.0
天润曲轴(002283)  False    True       4.0
巨人网络(002558)  False    True       5.0
[2018-05-09 07:53:57.595075] INFO: 回测: LONG LIST
[2018-05-09 07:53:57.599744] INFO: 回测: 
        

...

[2018-05-09 07:54:01.823580] INFO: 回测: LONG LIST
[2018-05-09 07:54:01.830812] INFO: 回测: 
              longs  shorts  sma_rank
乐视网(300104)    True   False    3394.0
杭州园林(300649)   True   False    3393.0
寒锐钴业(300618)   True   False    3392.0
安凯客车(000868)   True   False    3391.0
华菱钢铁(000932)   True   False    3390.0
[2018-05-09 07:54:01.836998] INFO: 回测: SHORT LIST
[2018-05-09 07:54:01.843254] INFO: 回测: 
              longs  shorts  sma_rank
运达科技(300440)  False    True       1.0
中文在线(300364)  False    True       2.0
朗科智能(300543)  False    True       3.0
唐人神(002567)   False    True       4.0
众应互联(002464)  False    True       5.0
[2018-05-09 07:54:01.844236] INFO: 回测: LONG LIST
[2018-05-09 07:54:01.851250] INFO: 回测: 
              longs  shorts  sma_rank
杭州园林(300649)   True   False    3396.0
乐视网(300104)    True   False    3395.0
安凯客车(000868)   True   False    3394.0
寒锐钴业(300618)   True   False    3393.0
国科微(300672)    True   False    3392.0
[2018-05-09 07:54:01.857641] INFO: 回测: SHORT LIS

[2018-05-09 07:54:02.115742] INFO: 回测: LONG LIST
[2018-05-09 07:54:02.121536] INFO: 回测: 
              longs  shorts  sma_rank
江丰电子(300666)   True   False    3419.0
安凯客车(000868)   True   False    3418.0
科恒股份(300340)   True   False    3417.0
中通国脉(603559)   True   False    3416.0
国科微(300672)    True   False    3415.0
[2018-05-09 07:54:02.129470] INFO: 回测: SHORT LIST
[2018-05-09 07:54:02.134783] INFO: 回测: 
               longs  shorts  sma_rank
华源控股(002787)   False    True       1.0
*ST中安(600654)  False    True       2.0
运达科技(300440)   False    True       3.0
中文在线(300364)   False    True       4.0
雪 莱 特(002076)  False    True       5.0
[2018-05-09 07:54:02.135653] INFO: 回测: LONG LIST
[2018-05-09 07:54:02.142630] INFO: 回测: 
              longs  shorts  sma_rank
江丰电子(300666)   True   False    3421.0
中通国脉(603559)   True   False    3420.0
安凯客车(000868)   True   False    3419.0
科恒股份(300340)   True   False    3418.0
国科微(300672)    True   False    3417.0
[2018-05-09 07:54:02.152615] INFO: 回测: SHO

[2018-05-09 07:54:02.426846] INFO: 回测: LONG LIST
[2018-05-09 07:54:02.433949] INFO: 回测: 
              longs  shorts  sma_rank
江丰电子(300666)   True   False    3437.0
三六零(601360)    True   False    3436.0
中通国脉(603559)   True   False    3435.0
鸿特精密(300176)   True   False    3434.0
华大基因(300676)   True   False    3433.0
[2018-05-09 07:54:02.441347] INFO: 回测: SHORT LIST
[2018-05-09 07:54:02.450814] INFO: 回测: 
               longs  shorts  sma_rank
华源控股(002787)   False    True       1.0
雪 莱 特(002076)  False    True       2.0
*ST中安(600654)  False    True       3.0
中文在线(300364)   False    True       4.0
运达科技(300440)   False    True       5.0
[2018-05-09 07:54:02.452086] INFO: 回测: LONG LIST
[2018-05-09 07:54:02.458882] INFO: 回测: 
              longs  shorts  sma_rank
三六零(601360)    True   False    3438.0
江丰电子(300666)   True   False    3437.0
中通国脉(603559)   True   False    3436.0
华大基因(300676)   True   False    3435.0
国科微(300672)    True   False    3434.0
[2018-05-09 07:54:02.466092] INFO: 回测: SHO

[2018-05-09 07:54:02.760530] INFO: 回测: LONG LIST
[2018-05-09 07:54:02.767458] INFO: 回测: 
              longs  shorts  sma_rank
三六零(601360)    True   False    3453.0
江丰电子(300666)   True   False    3452.0
韦尔股份(603501)   True   False    3451.0
士兰微(600460)    True   False    3450.0
华大基因(300676)   True   False    3449.0
[2018-05-09 07:54:02.773794] INFO: 回测: SHORT LIST
[2018-05-09 07:54:02.783178] INFO: 回测: 
               longs  shorts  sma_rank
华源控股(002787)   False    True       1.0
赛摩电气(300466)   False    True       2.0
雪 莱 特(002076)  False    True       3.0
中文在线(300364)   False    True       4.0
*ST中安(600654)  False    True       5.0
[2018-05-09 07:54:02.783916] INFO: 回测: LONG LIST
[2018-05-09 07:54:02.788306] INFO: 回测: 
              longs  shorts  sma_rank
三六零(601360)    True   False    3454.0
江丰电子(300666)   True   False    3453.0
韦尔股份(603501)   True   False    3452.0
士兰微(600460)    True   False    3451.0
华大基因(300676)   True   False    3450.0
[2018-05-09 07:54:02.794054] INFO: 回测: SHO

,algo_volatility,algorithm_period_return,alpha,benchmark_period_return,benchmark_volatility,beta,capital_used,ending_cash,ending_exposure,ending_value,...,short_exposure,short_value,shorts_count,sortino,starting_cash,starting_exposure,starting_value,trading_days,transactions,treasury_period_return
2017-01-03 07:00:00+00:00,NaN,0.0,NaN,0.009712,NaN,NaN,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,1,[],0.0
2017-01-04 07:00:00+00:00,0.0,0.0,0.0,0.017593,0.021406,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,2,[],0.0
2017-01-05 07:00:00+00:00,0.0,0.0,0.0,0.017435,0.083084,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,3,[],0.0
2017-01-06 07:00:00+00:00,0.0,0.0,0.0,0.011356,0.115404,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,4,[],0.0
2017-01-09 07:00:00+00:00,0.0,0.0,0.0,0.016261,0.100950,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,5,[],0.0
2017-01-10 07:00:00+00:00,0.0,0.0,0.0,0.014560,0.095760,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,6,[],0.0
2017-01-11 07:00:00+00:00,0.0,0.0,0.0,0.007377,0.104382,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,7,[],0.0
2017-01-12 07:00:00+00:00,0.0,0.0,0.0,0.002279,0.102578,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,8,[],0.0
2017-01-13 07:00:00+00:00,0.0,0.0,0.0,0.002971,0.095975,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,9,[],0.0
2017-01-16 07:00:00+00:00,0.0,0.0,0.0,0.002830,0.090519,0.0,0.0,10000000.0,0.0,0.0,...,0.0,0.0,0,None,10000000.0,0.0,0.0,10,[],0.0


**运行时间8.81秒**